In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
from torchvision.models import densenet161, densenet201, densenet121
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from datasets import load_metric

In [2]:
#путь к файлам
path_to_images = 'all_images'

train_csv = pd.read_csv('files_train.csv')
eval_csv = pd.read_csv('files_eval.csv')
test_csv = pd.read_csv('files_test.csv')

In [5]:
transform = transforms.Compose([
    torchvision.transforms.Resize(
        size=(224, 224)
    ),
    transforms.ToTensor(),
])

In [6]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_csv, dir_files, transform=None):
        self.dir_files = dir_files
        self.label = images_csv 
        self.transform = transform
        self.file_arr = np.array(self.label['files'].to_list())
        self.label_arr = np.array(self.label['classes'].to_list())
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        file_name = os.path.join(self.dir_files, str(self.file_arr[idx]))
        image = Image.open(file_name)
        image = image.convert('RGB')
        label = self.label_arr[idx]
        if self.transform:
            feature = self.transform(image)
        sample = {
            'pixel_values': feature, 
            'label': int(label),
            'file_name': file_name
        }
        return sample 

In [7]:
train_set = ImageDataset(images_csv = train_csv, dir_files = path_to_images, transform = transform)
eval_set = ImageDataset(images_csv = eval_csv, dir_files = path_to_images, transform = transform)
test_set = ImageDataset(images_csv = test_csv, dir_files = path_to_images, transform = transform)

In [8]:
print(len(train_set))
print(len(eval_set))
print(len(test_set))

2100
300
600


In [16]:
batch_size = 50

In [17]:
loader_train = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
loader_eval = torch.utils.data.DataLoader(eval_set, batch_size=batch_size, shuffle=True)
loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [10]:
model = densenet121()

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
device

device(type='cuda')

In [13]:
model = model.to(device)

In [15]:
num_epochs = 150
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(loader_train):
        # Move input and label tensors to the device
        inputs = batch["pixel_values"].to(device)
        labels = batch["label"].to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
    path = f'DenseNet_{epoch+1}.ckpt'
    torch.save(model.state_dict(), path)
    
    accuracy = load_metric("accuracy")
    model.eval()
    for batch_1 in tqdm(loader_eval):
        inputs = batch_1["pixel_values"].to(device)
        labels = batch_1["label"].to(device)
        
        outputs = model(inputs)
        predictions = outputs.argmax(-1).cpu().detach().numpy()
        _, predictions = torch.max(outputs.data, 1)
        references = batch_1["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)
    final_score = accuracy.compute()
    print("Accuracy on eval set:", final_score)

print(f'Finished Training, Loss: {loss.item():.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:26<00:00,  1.57it/s]


Epoch 1/150, Loss: 0.7244


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.60it/s]


Accuracy on eval set: {'accuracy': 0.6033333333333334}


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:27<00:00,  1.51it/s]


...

Accuracy on eval set: {'accuracy': 0.8533333333333334}


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:27<00:00,  1.50it/s]


Epoch 150/150, Loss: 0.0009


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.62it/s]

Accuracy on eval set: {'accuracy': 0.8333333333333334}
Finished Training, Loss: 0.0009


In [ ]:
model = densenet161()

In [ ]:
model = model.to(device)

In [ ]:
batch_size = 30

In [ ]:
loader_train = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
loader_eval = torch.utils.data.DataLoader(eval_set, batch_size=batch_size, shuffle=True)
loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [16]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(loader_train):
        # Move input and label tensors to the device
        inputs = batch["pixel_values"].to(device)
        labels = batch["label"].to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
    path = f'DenseNet_161_{epoch+1}.ckpt'
    torch.save(model.state_dict(), path)
    
    accuracy = load_metric("accuracy")
    model.eval()
    for batch_1 in tqdm(loader_eval):
        inputs = batch_1["pixel_values"].to(device)
        labels = batch_1["label"].to(device)
        
        outputs = model(inputs)
        predictions = outputs.argmax(-1).cpu().detach().numpy()
        _, predictions = torch.max(outputs.data, 1)
        references = batch_1["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)
    final_score = accuracy.compute()
    print("Accuracy on eval set:", final_score)

print(f'Finished Training, Loss: {loss.item():.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:45<00:00,  1.55it/s]


Epoch 1/150, Loss: 1.0392


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.18it/s]


Accuracy on eval set: {'accuracy': 0.5966666666666667}


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:46<00:00,  1.52it/s]


...

Accuracy on eval set: {'accuracy': 0.84}


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:44<00:00,  1.57it/s]


Epoch 150/150, Loss: 0.0015


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.13it/s]

Accuracy on eval set: {'accuracy': 0.8366666666666667}
Finished Training, Loss: 0.0015


In [12]:
model = densenet201()

In [13]:
model = model.to(device)

In [ ]:
batch_size = 20

In [ ]:
loader_train = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
loader_eval = torch.utils.data.DataLoader(eval_set, batch_size=batch_size, shuffle=True)
loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [16]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(loader_train):
        # Move input and label tensors to the device
        inputs = batch["pixel_values"].to(device)
        labels = batch["label"].to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
    path = f'DenseNet_201_{epoch+1}.ckpt'
    torch.save(model.state_dict(), path)
    
    accuracy = load_metric("accuracy")
    model.eval()
    for batch_1 in tqdm(loader_eval):
        inputs = batch_1["pixel_values"].to(device)
        labels = batch_1["label"].to(device)
        
        outputs = model(inputs)
        predictions = outputs.argmax(-1).cpu().detach().numpy()
        _, predictions = torch.max(outputs.data, 1)
        references = batch_1["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)
    final_score = accuracy.compute()
    print("Accuracy on eval set:", final_score)

print(f'Finished Training, Loss: {loss.item():.4f}')

100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:43<00:00,  2.40it/s]
C:\Users\Елизавета\AppData\Local\Temp\ipykernel_10760\4186076063.py:25: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy = load_metric("accuracy")


Epoch 1/150, Loss: 0.7954


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  5.79it/s]


Accuracy on eval set: {'accuracy': 0.55}


100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:36<00:00,  2.86it/s]


...

Accuracy on eval set: {'accuracy': 0.7266666666666667}


100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:38<00:00,  2.70it/s]


Epoch 150/150, Loss: 0.4047


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.27it/s]

Accuracy on eval set: {'accuracy': 0.7666666666666667}
Finished Training, Loss: 0.4047


In [9]:
model = densenet121()
model.load_state_dict(torch.load('DenseNet_79.ckpt'))

<All keys matched successfully>

In [13]:
model = model.to(device)

In [18]:
accuracy = load_metric("accuracy")
model.eval()
for batch in tqdm(loader_test):
    inputs = batch["pixel_values"].to(device)
    labels = batch["label"].to(device)
        
    outputs = model(inputs)
    predictions = outputs.argmax(-1).cpu().detach().numpy()
    _, predictions = torch.max(outputs.data, 1)
    references = batch["label"].numpy()
    accuracy.add_batch(predictions=predictions, references=references)
final_score = accuracy.compute()
print("Accuracy on test set:", final_score)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:32<00:00,  2.68s/it]


Accuracy on test set: {'accuracy': 0.8016666666666666}
